In [1]:
%load_ext autoreload
%autoreload 2

import os,sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath("../training"))

In [2]:
import os
os.environ["EXP_NAME"] = "movie_recom"
os.environ["MAX_RATING"] = "5"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:8081"
os.environ["EPOCH"] = "2"

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [3]:
from training.xgb import main, test_xgb_model
from movie_recommender.workflow import read_parquet_from_s3, connect_minio,connect_mlflow

DB_MINIO_BUCKET = "trainingbucket"
connect_minio()
connect_mlflow()

Secure False
endpoint http://localhost:9000
Connected to minio on: http://localhost:9000
Available Buckets [Bucket('mlflowbucket'), Bucket('trainingbucket')]
Connected to mlflow on: http://localhost:8081


In [4]:
ratings = read_parquet_from_s3(DB_MINIO_BUCKET, "ratings.parquet")
movies = read_parquet_from_s3(DB_MINIO_BUCKET, "movies.parquet")
ratings = ratings.merge(movies, on="movie_id")
ratings.rating *= 5

In [5]:
booster = main(ratings,n_trials=10)

[I 2025-09-20 11:58:48,772] A new study created in memory with name: no-name-292826d1-8269-4dc4-a600-d8aa4cc54309


Running CV...


  0%|          | 0/10 [00:00<?, ?it/s]

[0.805154   0.80137545 0.7591083  0.8116316  0.8245758  0.7021182
 0.7140878  0.706629   0.99238926 0.9925022 ]
Pred mean: 0.71 , std: 0.15
[0.7144053  0.796328   0.7153227  0.9034942  0.8554294  0.5204804
 0.7903971  0.6255897  0.73666865 0.7384317 ]
Pred mean: 0.70 , std: 0.15
[0.79828954 0.78987336 0.8530689  0.8045417  0.79974777 0.9935446
 0.60085666 0.8120242  0.73317313 0.80524397]
Pred mean: 0.71 , std: 0.15
[I 2025-09-20 11:58:58,929] Trial 0 finished with value: 0.14200381698904377 and parameters: {'objective': 'reg:squarederror', 'device': 'cuda', 'random_state': 0, 'eval_metric': 'mae', 'booster': 'gbtree', 'max_depth': 8, 'learning_rate': 0.03369051835914927, 'subsample': 0.6121776513465201, 'colsample_bytree': 0.9183124180871549, 'gamma': 0.3448541904469965, 'lambda': 0.01893940565537754, 'alpha': 0.004333934729622851}. Best is trial 0 with value: 0.14200381698904377.
[0.7936685  0.7911262  0.7529904  0.80201715 0.81250364 0.70227814
 0.7245163  0.7089904  0.9747189  0.97

[INFO] - Run id: 195c47d226294d6ca77a09dff63f6bfb


Found most impactful features: ['movie_avg_rating', 'mean_Action', 'mean_Thriller', 'mean_Drama', 'mean_Comedy', 'user_avg_rating']

Training XGBMR...
[0]	train-mae:0.16002	val-mae:0.15699
[500]	train-mae:0.09365	val-mae:0.13925
[550]	train-mae:0.09365	val-mae:0.13925


2025/09/20 12:00:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/20 12:00:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/phantom/miniconda3/envs/cudatorch/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [12:00:34] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
[INFO] - Logging artifacts.
[INFO] - Done training.


Logging folder /tmp/tmph5dnf_3e
🏃 View run secretive-sow-468 at: http://localhost:8081/#/experiments/1/runs/195c47d226294d6ca77a09dff63f6bfb
🧪 View experiment at: http://localhost:8081/#/experiments/1


Registered model 'xgb' already exists. Creating a new version of this model...
2025/09/20 12:00:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb, version 10
Created version '10' of model 'xgb'.


metrics {'train-mae': 0.10159176049064735, 'val-mae': 0.1373481480212674, 'stopped_iteration': 514.0, 'best_iteration': 14.0}
Champion 'val-mae': 0.1373, Latest 'val-mae': 0.1381
No new champion for 'xgb'

Saving plots...
Training
0.1524721 0.7124303
[0.6463927  0.7931963  0.66872764 ... 0.8126988  0.9734222  0.9016125 ]
True  : mean 3.562, std 1.054
Pred  : mean 3.562, std 0.762
MAE   : 0.4682349965935481
Error : mean 0.000, std 0.659
Test
0.14113747 0.70774657
[0.9713957  0.8450908  0.50933886 ... 0.7444314  0.6746332  0.7556304 ]
True  : mean 3.534, std 1.033
Pred  : mean 3.539, std 0.706
MAE   : 0.6962752185193702
Error : mean -0.005, std 0.943
Logging folder /tmp/tmp2q80nc9f


In [7]:
test_xgb_model()

XGB loaded on device: cuda
Getting artifacts, run_id='195c47d226294d6ca77a09dff63f6bfb', artifact_path='resources'


/home/phantom/git/n2ak/movie_recommender/backend/movie_recommender/workflow.py:114: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  return _mlflowClient.get_latest_versions(registered_name)[0].run_id


[INFO] - Loaded champion model, registered_name='xgb'


0.031598747 0.76028055
[0.78433657 0.76825815 0.7065908  0.7819369 ]


In [8]:
# from training.xgb import save_plots, XGBMR, prepare, simple_split, MovieLensPreprocessor
# model = XGBMR.load_model(champion=False, device="cuda")
# train, test = simple_split(ratings.copy(), .8)

In [9]:
# processor = MovieLensPreprocessor().fit(train)
# X_train, X_test, y_train, y_test = processor.do_transform(train, test)
# save_plots(
#     model,
#     prepare(X_train[model.cols], y_train),
#     prepare(X_test[model.cols], y_test),
#     max_rating=5,
#     run_id="26161cc8ad064fc18105ecb7a51da4ce",
# )